# Tesco Clubcard Data Analysis Notebook

This notebook serves to help me understand how the JSON file provided of Tesco Clubcard user data is structured and what various information can be drawn from it. I must pre-process, parse through, and identify key figures from the JSON data of transactions from a user's clubcard. 



## Imports

In [3]:
import json
import pandas as pd

## Load Transactions from JSON into DF

In [5]:
# Load raw JSON
with open("../../Downloads/Tesco-Customer-Data.json", "r") as json_file: 
    json_data = json.load(json_file)

# Extract Transactions
transactions = json_data["Purchase"][0]

#Create Trnasaction_DF
df_transactions = pd.json_normalize(transactions)
df_transactions.head()

,basketValueGross,purchaseType,overallBasketSavings,storeId,storeAddress,paymentType,timeStamp,basketValueNet,says,storeName,storeFormat,product
0,5.2,IN_STORE,1.35,4667,"Units 2&3 Distillers Building,Smithfield, Dubl...","[{'type': 'VisaCredit', 'category': 'NA', 'amo...",2024-12-15 17:17:32.514,5.2,NA,Smithfield Express,Express,[{'name': 'Tesco Slievenamon Irish Still Sprin...
1,4.74,IN_STORE,1.06,5980,51 - 52 Thomas Street,"[{'type': 'VisaCredit', 'category': 'NA', 'amo...",2024-12-12 16:04:56.304,0.74,NA,Thomas Street Express,Express,"[{'name': 'Kerrygold Butter Sticks 100G', 'qua..."
2,3.45,IN_STORE,0.7,5505,41 Upper Camden Street,"[{'type': 'VisaCredit', 'category': 'NA', 'amo...",2024-12-06 17:59:38.685,3.45,NA,Camden Street Express,Express,[{'name': 'Tesco Slievenamon Irish Still Sprin...
3,3.45,IN_STORE,0.7,5980,51 - 52 Thomas Street,"[{'type': 'VisaCredit', 'category': 'NA', 'amo...",2024-12-06 14:47:26.859,3.45,NA,Thomas Street Express,Express,[{'name': 'Tesco Slievenamon Irish Still Sprin...
4,4.9,IN_STORE,0.85,4667,"Units 2&3 Distillers Building,Smithfield, Dubl...","[{'type': 'VisaCredit', 'category': 'NA', 'amo...",2024-11-28 19:40:34.480,4.9,NA,Smithfield Express,Express,[{'name': 'Cadbury Dairy Milk Whole Nut Chocol...


## Initial Data Analysis

In [6]:
# Convert timeStamp column to datetime and handle that some are formatted differently
df_transactions["timeStamp"] = pd.to_datetime(df_transactions["timeStamp"], format="mixed", utc=True)

In [7]:
print("First Transaction Date:", df_transactions["timeStamp"].min())
print("Last Transaction Date:", df_transactions["timeStamp"].max())
print("Number of Transactions:", len(df_transactions))
print("Account Open for %d days" % ((df_transactions["timeStamp"].max() - df_transactions["timeStamp"].min()).days))
print("Visited %d different stores." % (len(df_transactions["storeId"].unique())))


First Transaction Date: 2019-09-11 18:46:52+00:00
Last Transaction Date: 2024-12-15 17:17:32.514000+00:00
Number of Transactions: 250
Account Open for 1921 days
Visited 17 different stores.


In [8]:
# View all the different stores visited
df_transactions["storeName"].unique()

array(['Smithfield Express', 'Thomas Street Express',
       'Camden Street Express', 'Crumlin Express', 'Castlebar Superstore',
       'Spencer Dock Express', 'Temple Bar Express',
       'Baggot Street Upper Metro', 'Dorset Street Express',
       'Cabra Superstore', 'Donnybrook Express', 'Bloomfields Superstore',
       'Parnell Street Metro', 'Merrion Superstore', 'BLOOMFIELDS',
       'CABRA', 'FINGLAS CLEARWATER', 'MERRION', 'TEMPLE BAR METRO',
       'THOMAS STREET EXPRESS', 'DONNYBROOK EXPRESS', 'STILLORGAN',
       'TALBOT STREET EXPRESS'], dtype=object)

### 

In [9]:
# Fix issue of Cabra listed under differet names 
df_transactions["storeName"] = df_transactions["storeName"].replace(
    {"CABRA": "Cabra Superstore"}
)

In [10]:
# Top 5 stores visited and # of times
df_transactions["storeName"].value_counts().head().reset_index(name="visits")

,storeName,visits
0,MERRION,69
1,Cabra Superstore,67
2,Thomas Street Express,43
3,Smithfield Express,22
4,STILLORGAN,9


In [11]:
# Save Cleaned Dataset 
df_transactions.to_json("../Tesco-Customer-Data-CLEANED.json", orient="records", indent=4)

## Transaction Analysis

In [12]:
# Create DF of items
items_rows = []

for _, tx in df_transactions.iterrows():
    for item in tx["product"]:
        items_rows.append({
            "transaction_time": tx["timeStamp"],
            "store": tx["storeName"],
            "item": item["name"],
            "quantity": int(item["quantity"]),
            "price": float(item["price"])
        })
df_items = pd.DataFrame(items_rows)
df_items

,transaction_time,store,item,quantity,price
0,2024-12-15 17:17:32.514000+00:00,Smithfield Express,Tesco Slievenamon Irish Still Spring Water 2 L...,1,0.95
1,2024-12-15 17:17:32.514000+00:00,Smithfield Express,Cadbury Dairy Milk Chocolate Whole Nut Bar 120G,1,1.75
2,2024-12-15 17:17:32.514000+00:00,Smithfield Express,Keogh's Crinkle Cut Crisps Salt&Vinegar125g,1,2.50
3,2024-12-12 16:04:56.304000+00:00,Thomas Street Express,Kerrygold Butter Sticks 100G,1,0.20
4,2024-12-12 16:04:56.304000+00:00,Thomas Street Express,Tesco Slievenamon Irish Still Spring Water 2 L...,1,0.15
...,...,...,...,...,...
2342,2019-09-11 18:46:52+00:00,MERRION,Tesco Health Chicken Curry And Rice 350G,1,1.50
2343,2019-09-11 18:46:52+00:00,MERRION,Tesco Fine Bean 200G,1,1.15
2344,2019-09-11 18:46:52+00:00,MERRION,Tesco Kitchen Towel White 4 Roll,1,1.19
2345,2019-09-11 18:46:52+00:00,MERRION,Tesco Fusilli Pasta 500G,1,0.49


In [13]:
df_items.groupby('item')['quantity'].sum().sort_values(ascending=False).head(30)

item
Tesco Irish Beef Sirloin Steak 266G                       173
Tesco Microwave Pilau Rice 250G                            75
Tesco Bananas Loose                                        62
Tesco Hot Salsa 300G                                       60
Tesco Gala Apple 7 Pack                                    59
Tesco Mini Carrot 320G                                     57
Tesco Fresh Potato Gnocchi Pasta 400 G                     54
Tesco Lightly Salted Tortilla Chips 200G                   48
Tesco Spinach 90G                                          46
Tesco Egg Rice 250G                                        41
Hearty Food Co. Chicken Flavour Noodles 65G                41
Glenisk Fresh Goats Milk 1 Litre                           39
Springforce Jumbo Kitchen Towel 2 Roll                     36
Tesco Mixed Peppers 3 Pack                                 33
Tesco Irish Smoked Streaky Rashers 253G                    33
Tesco Family Pack Onions 1 Kilograms                       32
Tes

In [14]:
# Mapping to fix items that are the same but listed under different Titles
mapping = {
    "Tesco Irish Beef Sirloin Steak 266G": "Tesco Beef Sirloin Steak 266G",
    "Tesco Fire Pit Beef Sirloin Steak 266g": "Tesco Beef Sirloin Steak 266G",
}
df_items["item"] = df_items["item"].replace(mapping)


In [16]:
df_items.groupby('item')['quantity'].sum().sort_values(ascending=False).head(50)

item
Tesco Beef Sirloin Steak 266G                                  194
Tesco Microwave Pilau Rice 250G                                 75
Tesco Bananas Loose                                             62
Tesco Hot Salsa 300G                                            60
Tesco Gala Apple 7 Pack                                         59
Tesco Mini Carrot 320G                                          57
Tesco Fresh Potato Gnocchi Pasta 400 G                          54
Tesco Lightly Salted Tortilla Chips 200G                        48
Tesco Spinach 90G                                               46
Hearty Food Co. Chicken Flavour Noodles 65G                     41
Tesco Egg Rice 250G                                             41
Glenisk Fresh Goats Milk 1 Litre                                39
Springforce Jumbo Kitchen Towel 2 Roll                          36
Tesco Irish Smoked Streaky Rashers 253G                         33
Tesco Mixed Peppers 3 Pack                               